In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

### 1. Loading data

In [3]:
raw_df = pd.read_csv("./data/hotel_bookings_modified.csv", low_memory=False)
df = raw_df.copy()

In [4]:
df.dtypes

hotel                              object
is_canceled                         int64
lead_time                           int64
arrival_date_year                 float64
arrival_date_month                 object
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                          float64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes                     int64
deposit_type                       object
agent                             

In [5]:
df.sample(5)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,kids
56357,City Hotel,1,167,2016.0,September,36,3,2,5,2,0.0,0,BB,FRA,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9.0,NaN,0.0,Transient,119.85,0.0,0.0,Canceled,2016-03-22,NaN
13252,Resort Hotel,1,126,2017.0,August,32,6,2,3,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,242.0,NaN,0.0,Transient,239.00,0.0,1.0,Canceled,2017-04-05,NaN
34607,Resort Hotel,0,0,2017.0,March,13,30,0,1,1,0.0,0,BB,PRT,Corporate,Direct,0,0,0,A,A,1,No Deposit,NaN,225.0,0.0,Transient,35.00,0.0,0.0,Check-Out,2017-03-31,NaN
52664,City Hotel,1,337,2016.0,June,24,9,0,2,2,0.0,0,BB,PRT,Groups,TA/TO,0,0,0,A,A,0,Non Refund,1.0,NaN,0.0,Transient,62.00,0.0,0.0,Canceled,2015-10-21,NaN
15932,Resort Hotel,0,51,2015.0,July,31,29,2,8,2,0.0,0,BB,NLD,Online TA,TA/TO,0,0,0,E,E,0,No Deposit,240.0,NaN,0.0,Transient,167.80,1.0,0.0,Check-Out,2015-08-08,NaN


### 2. Cleaning of values column by column

In [6]:
columns_to_clean = list(df.columns)
print(columns_to_clean)

['hotel', 'is_canceled', 'lead_time', 'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number', 'arrival_date_day_of_month', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children', 'babies', 'meal', 'country', 'market_segment', 'distribution_channel', 'is_repeated_guest', 'previous_cancellations', 'previous_bookings_not_canceled', 'reserved_room_type', 'assigned_room_type', 'booking_changes', 'deposit_type', 'agent', 'company', 'days_in_waiting_list', 'customer_type', 'adr', 'required_car_parking_spaces', 'total_of_special_requests', 'reservation_status', 'reservation_status_date', 'kids']


In [7]:
assert set(df["hotel"].unique()) == set(['Resort Hotel', 'City Hotel'])
columns_to_clean.remove("hotel")

assert set(df["is_canceled"].unique()) == set([0, 1])
columns_to_clean.remove("is_canceled")

assert np.max(df["lead_time"].unique()) < 1000
columns_to_clean.remove("lead_time")

In [8]:
df["arrival_date_year"].value_counts(dropna=False)

arrival_date_year
2016.0     30105
2015.0     14537
2017.0     13051
20016.0      614
NaN          588
Name: count, dtype: int64

In [9]:
df = df.dropna(subset=["arrival_date_year"])
df["arrival_date_year"] = df["arrival_date_year"].astype(int)
df["arrival_date_year"] = df["arrival_date_year"].replace(20016, 2016)

In [10]:
df["arrival_date_year"].value_counts(dropna=False)

arrival_date_year
2016    30719
2015    14537
2017    13051
Name: count, dtype: int64

In [11]:
assert set(df["arrival_date_year"].unique()) == set([2015, 2016, 2017])
columns_to_clean.remove("arrival_date_year")

assert len(df["arrival_date_month"].unique()) <= 12
columns_to_clean.remove("arrival_date_month")

for week_number in df["arrival_date_week_number"].unique():
    assert week_number in range(54)
columns_to_clean.remove("arrival_date_week_number")

for week_number in df["arrival_date_day_of_month"].unique():
    assert week_number in range(32)
columns_to_clean.remove("arrival_date_day_of_month")

assert np.min(df["stays_in_weekend_nights"].unique()) >= 0
columns_to_clean.remove("stays_in_weekend_nights")

assert np.min(df["stays_in_week_nights"].unique()) >= 0
columns_to_clean.remove("stays_in_week_nights")

In [12]:
df["adults"].value_counts(dropna=False)

adults
 2      45258
 1      10505
 3       2201
 0        102
-1         97
 4         34
 66         6
 26         5
 65         5
 53         4
 69         4
 59         4
 54         4
 91         4
 84         3
 95         3
 100        3
 72         3
 76         3
 50         3
 20         2
 27         2
 60         2
 86         2
 98         2
 68         2
 51         2
 5          2
 55         2
 78         2
 83         2
 64         2
 56         2
 73         2
 80         2
 92         2
 96         2
 57         2
 62         2
 79         2
 67         2
 52         2
 6          1
 10         1
 40         1
 89         1
 58         1
 93         1
 63         1
 61         1
 87         1
 85         1
 77         1
 75         1
Name: count, dtype: int64

In [13]:
df = df[df["adults"] != -1]

assert np.min(df["adults"].unique()) >= 0
columns_to_clean.remove("adults")

In [14]:
df["children"].value_counts(dropna=False)

children
0.0     53786
1.0      2307
2.0      2086
3.0        26
NaN         4
10.0        1
Name: count, dtype: int64

In [15]:
df = df.dropna(subset=["children"])
df["children"] = df["children"].astype(int)

assert np.min(df["children"].unique()) >= 0
columns_to_clean.remove("children")

df["children"].value_counts(dropna=False)

children
0     53786
1      2307
2      2086
3        26
10        1
Name: count, dtype: int64

In [16]:
df["babies"].value_counts(dropna=False)

babies
 0      57411
 1        593
-1         90
 2          8
 51         6
 57         5
 73         5
 77         5
 94         4
 97         4
 81         4
 56         4
 65         4
 61         3
 67         3
 70         3
 92         3
 66         3
 93         3
 59         3
 72         2
 82         2
 58         2
 75         2
 55         2
 52         2
 69         2
 53         2
 96         2
 71         2
 98         2
 100        2
 63         2
 78         2
 99         2
 54         1
 89         1
 62         1
 95         1
 74         1
 60         1
 84         1
 10         1
 91         1
 50         1
 83         1
 88         1
Name: count, dtype: int64

In [17]:
df = df[df["babies"] != -1]

assert np.min(df["babies"].unique()) >= 0
columns_to_clean.remove("babies")

df["babies"].value_counts(dropna=False)

babies
0      57411
1        593
2          8
51         6
57         5
73         5
77         5
94         4
97         4
81         4
56         4
65         4
61         3
67         3
70         3
92         3
66         3
93         3
59         3
72         2
82         2
58         2
75         2
55         2
52         2
69         2
53         2
96         2
71         2
98         2
100        2
63         2
78         2
99         2
54         1
89         1
62         1
95         1
74         1
60         1
84         1
10         1
91         1
50         1
83         1
88         1
Name: count, dtype: int64

In [18]:
columns_to_clean.remove("meal")

df = df.dropna(subset=["country"])
columns_to_clean.remove("country")

columns_to_clean.remove("market_segment")

columns_to_clean.remove("distribution_channel")

assert set(df["is_repeated_guest"].unique()) == set([0, 1])
columns_to_clean.remove("is_repeated_guest")

assert np.min(df["previous_cancellations"].unique()) >= 0
columns_to_clean.remove("previous_cancellations")

assert np.min(df["previous_bookings_not_canceled"].unique()) >= 0
columns_to_clean.remove("previous_bookings_not_canceled")

columns_to_clean.remove("reserved_room_type")

columns_to_clean.remove("assigned_room_type")

columns_to_clean.remove("booking_changes")

In [19]:
df["deposit_type"].value_counts()

deposit_type
No Deposit    51190
Non Refund     5375
No Refund       943
Refundable      141
Name: count, dtype: int64

In [20]:
df['deposit_type'] = df['deposit_type'].replace("No Refund", "Non Refund")

columns_to_clean.remove("deposit_type")

df["deposit_type"].value_counts()

deposit_type
No Deposit    51190
Non Refund     6318
Refundable      141
Name: count, dtype: int64

In [21]:
df["agent"].value_counts(dropna=False)

agent
240.0    13682
NaN       8696
9.0       6910
1.0       3139
250.0     2819
         ...  
179.0        1
93.0         1
210.0        1
215.0        1
77.0         1
Name: count, Length: 249, dtype: int64

In [22]:
df["agent"] = df["agent"].fillna(0)
df["agent"] = df["agent"].astype(int)

assert np.min(df["agent"].unique()) >= 0
columns_to_clean.remove("agent")

df["agent"].value_counts(dropna=False)

agent
240    13682
0       8696
9       6910
1       3139
250     2819
       ...  
179        1
93         1
210        1
215        1
77         1
Name: count, Length: 249, dtype: int64

In [23]:
df["company"].value_counts(dropna=False)

company
NaN      54379
223.0      770
281.0      136
154.0      123
405.0      100
         ...  
309          1
331          1
93           1
316          1
NU           1
Name: count, Length: 273, dtype: int64

In [24]:
df = df[df["company"] != 'NU']
df["company"] = df["company"].fillna('0.0')
df["company"] = df["company"].astype(float)
df["company"] = df["company"].astype(int)

assert np.min(df["company"].unique()) >= 0
columns_to_clean.remove("company")

df["company"].value_counts(dropna=False)

company
0      54379
223      770
281      136
154      123
405      100
       ...  
126        1
64         1
242        1
93         1
316        1
Name: count, Length: 258, dtype: int64

In [25]:
df["days_in_waiting_list"].value_counts(dropna=False)

days_in_waiting_list
0.0      55290
39.0       184
58.0       162
31.0        99
69.0        89
87.0        78
63.0        76
111.0       69
101.0       65
77.0        62
223.0       60
3.0         59
62.0        58
50.0        57
122.0       54
91.0        49
176.0       48
187.0       45
48.0        43
75.0        40
93.0        39
60.0        39
35.0        38
236.0       35
65.0        33
68.0        29
98.0        28
33.0        28
41.0        28
160.0       25
20.0        25
57.0        24
120.0       23
47.0        23
80.0        22
21.0        21
96.0        21
108.0       20
215.0       20
79.0        20
24.0        19
70.0        18
38.0        18
99.0        17
162.0       17
32.0        17
125.0       16
207.0       15
330.0       15
379.0       15
174.0       15
85.0        15
391.0       14
49.0        13
40.0        11
150.0       11
55.0        10
34.0        10
4.0         10
224.0       10
259.0       10
147.0        9
59.0         8
56.0         8
14.0         7
71.0

In [26]:
df = df.dropna(subset=["days_in_waiting_list"])
df["days_in_waiting_list"] = df["days_in_waiting_list"].astype(int)

assert np.min(df["days_in_waiting_list"].unique()) >= 0
columns_to_clean.remove("days_in_waiting_list")

df["days_in_waiting_list"].value_counts(dropna=False)

days_in_waiting_list
0      55290
39       184
58       162
31        99
69        89
87        78
63        76
111       69
101       65
77        62
223       60
3         59
62        58
50        57
122       54
91        49
176       48
187       45
48        43
75        40
93        39
60        39
35        38
236       35
65        33
68        29
98        28
33        28
41        28
160       25
20        25
57        24
120       23
47        23
80        22
21        21
96        21
108       20
215       20
79        20
24        19
70        18
38        18
99        17
162       17
32        17
125       16
207       15
330       15
379       15
174       15
85        15
391       14
49        13
40        11
150       11
55        10
34        10
4         10
224       10
259       10
147        9
59         8
56         8
14         7
71         7
15         7
1          6
8          6
113        5
107        5
53         5
178        5
6          4
5          4
97  

In [27]:
df["customer_type"].value_counts(dropna=False)

customer_type
Transient          41465
Transient-Party    12847
Contract            2459
NaN                  575
Group                302
Name: count, dtype: int64

In [28]:
df = df.dropna(subset=["customer_type"])

columns_to_clean.remove("customer_type")

df["customer_type"].value_counts(dropna=False)

customer_type
Transient          41465
Transient-Party    12847
Contract            2459
Group                302
Name: count, dtype: int64

In [29]:
df["adr"].value_counts(dropna=False)

adr
62.00     1752
75.00     1046
48.00      996
0.00       920
65.00      896
          ... 
136.89       1
103.46       1
159.17       1
202.54       1
148.31       1
Name: count, Length: 6675, dtype: int64

In [30]:
df = df[df["adr"] != -6.38]
df = df[df["adr"] != 5400]

assert np.min(df["adr"].unique()) >= 0
columns_to_clean.remove("adr")

df["adr"].value_counts(dropna=False)

adr
62.00     1752
75.00     1046
48.00      996
0.00       920
65.00      896
          ... 
103.06       1
211.86       1
91.51        1
132.45       1
152.81       1
Name: count, Length: 6673, dtype: int64

In [31]:
df["required_car_parking_spaces"].value_counts(dropna=False)

required_car_parking_spaces
0.0    51124
1.0     5349
NaN      571
2.0       24
8.0        2
3.0        1
Name: count, dtype: int64

In [32]:
df["required_car_parking_spaces"] = df["required_car_parking_spaces"].fillna(0)
df["required_car_parking_spaces"] = df["required_car_parking_spaces"].astype(int)

assert np.min(df["required_car_parking_spaces"].unique()) >= 0
columns_to_clean.remove("required_car_parking_spaces")

df["required_car_parking_spaces"].value_counts(dropna=False)


required_car_parking_spaces
0    51695
1     5349
2       24
8        2
3        1
Name: count, dtype: int64

In [33]:
df["total_of_special_requests"] = df["total_of_special_requests"].astype(int)

assert np.min(df["total_of_special_requests"].unique()) >= 0
columns_to_clean.remove("total_of_special_requests")

df["total_of_special_requests"].value_counts(dropna=False)

total_of_special_requests
0    35970
1    14282
2     5633
3     1032
4      143
5       11
Name: count, dtype: int64

In [34]:
df["reservation_status"].value_counts(dropna=False)

reservation_status
Check-Out    33110
Canceled     22614
No-Show        777
NaN            570
Name: count, dtype: int64

In [35]:
df = df.dropna(subset=["reservation_status"])

columns_to_clean.remove("reservation_status")

df["reservation_status"].value_counts(dropna=False)

reservation_status
Check-Out    33110
Canceled     22614
No-Show        777
Name: count, dtype: int64

In [36]:
columns_to_clean.remove("reservation_status_date")

In [37]:
df = df.drop("kids", axis=1)
columns_to_clean.remove("kids")

In [38]:
len(columns_to_clean)

0

### 3. Display the result

In [39]:
raw_df.shape, df.shape

((58895, 33), (56501, 32))

In [40]:
raw_df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,kids
0,Resort Hotel,0,342,2015.0,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01,NaN
1,Resort Hotel,0,737,2015.0,July,27,1,0,0,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01,NaN
2,Resort Hotel,0,7,2015.0,July,27,1,0,1,1,0.0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,NaN,NaN,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02,NaN
3,Resort Hotel,0,13,2015.0,July,27,1,0,1,1,0.0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304.0,NaN,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02,NaN
4,Resort Hotel,0,14,2015.0,July,27,1,0,2,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0.0,Transient,98.0,0.0,1.0,Check-Out,2015-07-03,NaN


In [41]:
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,0,0,BB,PRT,Direct,Direct,0,0,0,C,C,3,No Deposit,0,0,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,0,0,BB,PRT,Direct,Direct,0,0,0,C,C,4,No Deposit,0,0,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,0,0,BB,GBR,Direct,Direct,0,0,0,A,C,0,No Deposit,0,0,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,0,0,BB,GBR,Corporate,Corporate,0,0,0,A,A,0,No Deposit,304,0,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240,0,0,Transient,98.0,0,1,Check-Out,2015-07-03
